In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from datetime import datetime, timedelta, timezone

spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext

In [2]:
utc_minus_3 = timezone(timedelta(hours=-3))

_year = datetime.now(utc_minus_3).strftime("%Y")
_month = datetime.now(utc_minus_3).strftime("%m")
_day = datetime.now(utc_minus_3).strftime("%d")

In [3]:
df_bronzeBreweries = spark.read.option("multiLine", False).json(f'../datalake/bronze/breweries/{_year}/{_month}/{_day}/breweries.json.gz')

In [4]:
df_silverBreweries = (
    df_bronzeBreweries
    .select(
        F.col('id').cast('string'),
        F.col('name').cast('string'),
        F.col('brewery_type').cast('string'),
        F.col('website_url').cast('string'),
        F.col('phone').cast('string'),
        F.col('postal_code').cast('string'),
        F.col('state_province').cast('string'),
        F.col('country').cast('string'),
        F.col('state').cast('string'),
        F.col('city').cast('string'),
        F.col('street').cast('string'),
        F.col('longitude').cast('string'),
        F.col('latitude').cast('string'),
        F.col('address_1').cast('string'),
        F.col('address_2').cast('string'),
        F.col('address_3').cast('string')
    )    
)

In [5]:
(
    df_silverBreweries
    .coalesce(1)
    .write
    .partitionBy("country")
    .mode('overwrite')
    .parquet(f"../datalake/silver/breweries/")
)